# Дата цэвэрлэх

In [479]:
# Хэрэгтэй сангуудыг оруулах
import pandas as pd
import re
import os
from datetime import datetime, timedelta

In [480]:
main_path = 'input'
df = pd.read_csv(main_path + '/ads_data_2025-06-18_13-59-54.csv')

### Хэрэггүй багана арилга

In [481]:
df.drop(columns=['page_number','ad_number','phone', 'Төлбөрийн нөхцөл:'], inplace=True)

### Хоосон утгыг устгах

In [482]:
df.isna().sum()

url               0
title             0
location          0
date              0
id                0
price             0
ad_text           0
Талбай:          52
Барилгын явц:    52
clean_coords     10
dtype: int64

In [483]:
df.dropna(subset=['Талбай:', 'Барилгын явц:'], inplace=True )

### Давхацсан утгыг устгах

In [484]:
df[df.duplicated(subset=['id', 'title', 'date'], keep=False)].count()

url              14
title            14
location         14
date             14
id               14
price            14
ad_text          14
Талбай:          14
Барилгын явц:    14
clean_coords     14
dtype: int64

In [485]:
df.drop_duplicates(subset=['id', 'title', 'date'], keep='first',inplace=True)

### Title болон ad_text баганад байгаа оффисын нэрийг нэг баганад нэмэх

In [486]:
# name.xlsx-ийг ачаалж, dictionary болгох
replace_df = pd.read_excel('D:/Projects/Office/02_data_cleaning/input/name.xlsx', header=1)
replace_df.columns = ['title', 'mytitle']  
replace_dict = dict(zip(replace_df['title'], replace_df['mytitle']))

In [487]:
# Функц: title болон ad_text баганыг шалгаж, тохирсон mytitle-г буцаана
def find_office_name(row):
    texts = []
    if isinstance(row['title'], str):
        texts.append(row['title'].lower())
    if isinstance(row['ad_text'], str):
        texts.append(row['ad_text'].lower())

    for key, value in replace_dict.items():
        key_lower = str(key).lower()
        if any(key_lower in text for text in texts):
            return value
    return None

In [488]:
df['mytitle'] = df.apply(find_office_name, axis=1)
df = df.rename(columns={'mytitle' : 'office_name'})

### Хороо дүүрэг салгах

In [489]:
import util as ut 

In [490]:
 #District name
df['district'] = df['location'].apply(ut.get_district)

In [491]:
#Khoroo
df['khoroo'] = df['location'].apply(ut.get_khoroo)

### Байршил ойролцоогоор хаяглах

In [492]:
# location 
df_loc = pd.read_csv('D:/Projects/Office/02_data_cleaning/input/location.csv')

In [493]:
df = df.merge(df_loc, how='left', on='location')

df['mylocation'].value_counts()
df['mylocation'].isna().sum()


0

In [494]:
df[df['mylocation'].isna()][['location','mylocation']]

Empty DataFrame
Columns: [location, mylocation]
Index: []

### Зарим баганы нэрийг өөрчлөх

In [495]:
name_cols = {
           'Title' : 'title',
            'price' : 'price_total',
            'Date'  : 'date',
            'Time'  : 'time',
            'id'    : 'ad_id',
            'loc'   : 'location',
            'message' : 'ad_text',
            'Талбай:' : 'size',
            'Барилгын явц:'          : 'progress_cons',
            'Төлбөрийн нөхцөл:'      : 'payment_terms',
            }

df.rename(columns=name_cols, inplace=True)
df.columns

Index(['url', 'title', 'location', 'date', 'ad_id', 'price_total', 'ad_text',
       'size', 'progress_cons', 'clean_coords', 'office_name', 'district',
       'khoroo', 'mylocation'],
      dtype='object')

### Талбайн болон үнийн мэдээллийг цэвэрлэж тоо болгох

In [496]:
df[['price_total','size']]

price_total       size
0                  482.04 сая ₮      74 м²
1                  254.41 сая ₮      39 м²
2                     7.3 сая ₮      65 м²
3     1.4 Тэрбум ₮\nүнэ тохирно     210 м²
4       175 сая ₮\n193.95 сая ₮      34 м²
...                         ...        ...
1082              1.26 Тэрбум ₮  200 м² м²
1083                  7.5 сая ₮     107 м²
1084                  290 сая ₮      56 м²
1085                  4.2 сая ₮     220 м²
1086                  8.9 сая ₮     197 м²

[1087 rows x 2 columns]

### Талбайн хэмжээ

In [497]:
df['area'] = df['size'].apply(
    lambda x: float(re.findall(r'\d+[\.\d]*', str(x))[0]) if pd.notnull(x) and re.findall(r'\d+[\.\d]*', str(x)) else None
)
df[['area','size']]
df[['area','size']].dtypes


area    float64
size     object
dtype: object

In [498]:
df = df[df['area'] >= 10 ]

### Оффисын үнэ

Хэрэггүй утгыг арилгаж Float болгох

In [499]:
df['price'] = df['price_total'].apply(lambda x: re.findall(r'\d+[\.\d]*', x)[0]).astype(float)
df[['price','price_total']]
df[['price','price_total']].dtypes

price          float64
price_total     object
dtype: object

Тэр бум гэсэн утгыг тоо болгох

In [500]:
mask = df['price_total'].str.contains('бум', na=False) & (df['price'] < 16)
df.loc[mask, 'price'] = df.loc[mask, 'price'] * 1000

In [501]:
df['price'].describe()
df.sort_values(by='price', ascending=False)

url  \
515   https://www.unegui.mn/adv/8411374_ondor-zeregl...   
1026  https://www.unegui.mn/adv/9330971_udirdlgyn-ak...   
610   https://www.unegui.mn/adv/9355924_khotyn-aaa-b...   
57    https://www.unegui.mn/adv/9369577_bzd-khavdar-...   
306   https://www.unegui.mn/adv/8733135_skhd-dragony...   
...                                                 ...   
566   https://www.unegui.mn/adv/9232564_tets-3-yn-ba...   
628   https://www.unegui.mn/adv/8920965_khud-3-r-kho...   
791   https://www.unegui.mn/adv/9355976_21-horoolold...   
699   https://www.unegui.mn/adv/8734912_dolgoon-nuur...   
634   https://www.unegui.mn/adv/9151717_3-4-khoroolo...   

                                                  title  \
515                       Өндөр зэрэглэлийн оффис таурт   
1026          Удирдлагын академи баруун талд титан таур   
610                       Хотын төвд оффис 850мк талбай   
57    Бзд, хавдар судлалын буудал, gumuda plaza 960м...   
306                               Схд драгоны ард оффис   
...                                                 ...   
566             Тэц 3-ын баруун талд оффис 135мк талбай   
628                          Худ 3-р хороо оффис 135мкв   
791                        21-r horoolold 141mkv office   
699   Сбд дөлгөөн нуурын зүүн талд computer mall офф...   
634               Бгд 3, 4 хороололд оффис 120мк талбай   

                                 location                         date  \
515    УБ — Сүхбаатар, Сүхбаатар, Хороо 1  Нийтэлсэн: 2025-06-09 22:06   
1026     УБ — Хан-Уул, Удирдлагын Академи  Нийтэлсэн: 2025-05-22 09:26   
610    УБ — Сүхбаатар, Сүхбаатар, Хороо 1  Нийтэлсэн: 2025-06-08 16:12   
57           УБ — Баянзүрх, 13-р хороолол     Нийтэлсэн: Өнөөдөр 10:54   
306            УБ — Сонгинохайрхан, 5 шар  Нийтэлсэн: 2025-06-14 11:52   
...                                   ...                          ...   
566   УБ — Хан-Уул, Үйлдвэрийн баруун бүс  Нийтэлсэн: 2025-06-09 12:34   
628        УБ — Хан-Уул, Хан-Уул, Хороо 3  Нийтэлсэн: 2025-06-07 16:04   
791    УБ — Сонгинохайрхан, 21-р хороолол  Нийтэлсэн: 2025-06-03 16:33   
699    УБ — Сүхбаатар, Сүхбаатар, Хороо 9  Нийтэлсэн: 2025-06-05 12:22   
634           УБ — Баянгол, 3, 4 хороолол  Нийтэлсэн: 2025-06-07 11:32   

        ad_id                 price_total  \
515   8411374                 10 Тэрбум ₮   
1026  9330971                8.1 Тэрбум ₮   
610   9355924                7.5 Тэрбум ₮   
57    9369577  6.72 Тэрбум ₮\nүнэ тохирно   
306   8733135                6.5 Тэрбум ₮   
...       ...                         ...   
566   9232564                   2.8 сая ₮   
628   8920965     2.65 сая ₮\nүнэ тохирно   
791   9355976        2.6 сая ₮\n2.8 сая ₮   
699   8734912      2.5 сая ₮\nүнэ тохирно   
634   9151717      2.4 сая ₮\nүнэ тохирно   

                                                ad_text     size  \
515   📍Өндөр зэрэглэлийн оффис таурт өндөр давхарт, ...   545 м²   
1026  TITAN TOWER-т оффис үйлчилгээний талбай худ...   900 м²   
610   📍"Аюуд" тауэрт давхраар нь оффисийн талбай зар...   850 м²   
57    БЗД, Хавдар судлалын буудал, Gumuda plaza 960м...   960 м²   
306   3 давхар оффис, гарааш том төвийн дулаан цахил...  2800 м²   
...                                                 ...      ...   
566   ХУД ТЭЦ-3 арын зам дагуу 5 давхар барилгын 5 д...   135 м²   
628   📣135м2 #Оффис, Үйлчилгээний талбай худалдана.\...   135 м²   
791                         21-r horoolold office zarna   141 м²   
699   ✨Дөлгөөн нуурын зүүн талд Computer malld оффис...   434 м²   
634   💎 3.4 хороолол дүүхээ төвийн урд  ажил оффис ч...   120 м²   

           progress_cons                       clean_coords   office_name  \
515     Ашиглалтад орсон               47.912432 106.921754          None   
1026  Ашиглалтад ороогүй                47.897313 106.91261       N tower   
610     Ашиглалтад орсон               47.912432 106.921754          None   
57    Ашиглалтад ороогүй               47.913249 106.938278  Gumuda plaza   
30

1 м.кв-ийн үнийг бодож гаргах

In [502]:
df = df[(df['price'] < 15) | (df['price'] > 50)]  # 15-с 50-ийн хоорондох утгыг хасах

In [503]:
df['price_m2'] = df['price']

In [504]:
mask = df['price'] > 15 #15-с их утгуудыг маск болгож авна. 
df.loc[mask, 'price_m2'] = df.loc[mask, 'price'] / df.loc[mask, 'area'] #15-с их утгыг талбайд нь хувааж 1 м.кв үнийг гаргана
df['price_m2'] = df['price_m2'].round(2) # 10-ны орноор бүхэлчлэв.

1 м.кв-ийн үнээр байгааг нийт үнэ болгох

In [505]:
mask = df['price'] < 15 #15-с их утгуудыг маск болгож авна. 
df.loc[mask, 'price'] = df.loc[mask, 'price_m2'] * df.loc[mask, 'area'] # 1 м.кв-аар байгаа үнийг нийт үнэ болгох
df['price'] = df['price'].round(2) # 10-ны орноор бүхэлчлэв.

In [506]:
df.sort_values(by='price_m2', ascending=False)[['office_name','price', 'area', 'price_m2', 'ad_id']]

office_name    price   area  price_m2    ad_id
515            None  10000.0  545.0     18.35  8411374
49   Minister Tower   1743.3  117.0     14.90  8999119
376  Minister Tower   1740.0  117.0     14.87  9441646
934     River plaza    670.0   47.0     14.26  9239644
622  Minister Tower   1630.0  117.0     13.93  8494225
..              ...      ...    ...       ...      ...
641            None    130.0   60.0      2.17  9421755
96             None     80.0   53.0      1.51  9456693
91             None     80.0   57.0      1.40  9457079
717            None     77.2  285.0      0.27  9415257
281            None    130.0  680.0      0.19  9139226

[1082 rows x 5 columns]

### Огноог хувиргах

In [507]:
import pandas as pd
from datetime import datetime, timedelta

# Одоогийн огноог авна (2025-06-20 14:09 гэж авч байна)
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
yesterday = today - timedelta(days=1)

# Огноог хөрвүүлэх функц
def parse_date(text):
    date_str = text.replace("Нийтэлсэн: ", "").strip()
    if date_str.startswith("Өнөөдөр"):
        time_str = date_str.replace("Өнөөдөр ", "").strip()
        return datetime.strptime(f"{today.strftime('%Y-%m-%d')} {time_str}", "%Y-%m-%d %H:%M")
    elif date_str.startswith("Өчигдөр"):
        time_str = date_str.replace("Өчигдөр ", "").strip()
        return datetime.strptime(f"{yesterday.strftime('%Y-%m-%d')} {time_str}", "%Y-%m-%d %H:%M")
    else:
        return datetime.strptime(date_str, "%Y-%m-%d %H:%M")

# Огноог хөрвүүлэх
df["parsed_date"] = df["date"].apply(parse_date)
# Огноо, цагийг тусад нь баганад оруулах
df["date"] = df["parsed_date"].dt.strftime("%Y-%m-%d")
df["time"] = df["parsed_date"].dt.strftime("%H:%M")

### Хэрэггүй багана устгаж файл хадгалах

Хэрэггүй багана устгах

In [509]:
df.drop(columns=['location', 'price_total', 'size', 'parsed_date'], inplace=True)

Оффисын нэр, нийт үнэ, талбайн хэмжээ давхацсан заруудыг устгах

In [511]:
df2.drop_duplicates(subset=['office_name', 'price', 'area'], keep='first', inplace=True)

C:\Users\Rentsen.Ts\AppData\Local\Temp\ipykernel_4888\2426734457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop_duplicates(subset=['office_name', 'price', 'area'], keep='first', inplace=True)


Зөв дарааллаар Dataframe үүсгэх

In [510]:
df2 = df[['ad_id', 'district', 'khoroo', 'mylocation','office_name','price', 'area', 'price_m2', 'progress_cons','title', 'ad_text', 'clean_coords','url', 'date', 'time']]

Dataframe хадгалах

In [512]:
# 'result' фолдер үүсгэх
folder_path = "result"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Файлыг 'result' фолдерт хадгалах
file_path = os.path.join(folder_path, "output.csv")
df2.to_csv(file_path, index=False,  encoding='utf-8-sig')